In [4]:
# Auto-encoder for 2D cortex slices
# Chuong Nguyen
# 07/11/2017
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import tensorflow as tf
import numpy as np
import h5py
import glob
import os
import argparse
import sys
from datetime import datetime
from pkg_resources import parse_version

# import local libraries
cd /data1/CHUONG_DATA/ChuongWork/Data4DeepLearning/src/
from eeg_input_data import eeg_data
from eeg_org_data import eeg_subject_data
import models
from utils import get_input_data_path, get_data_path_with_timestamp
from utils import build_model
from models.fc_freqSum_TiedWeight import build_fc_freqSum_TiedWeight
from models.fc_freqSum_TiedWeight import build_fc_freqSum_TiedWeight_NoBias
from models.fc_freqSum_TiedWeight import build_fc_freqSum_TiedWeight_NoDropout
from models.fc_freqSum_TiedWeight import build_fc_freqSum
from models.fc_freqSum_TiedWeight import build_fc_freqSum_TiedWeight_Big
from models.fc_freqSum_TiedWeight import build_fc_freqSum_NoTiedWeight_Big
from models.fc_freqSum_TiedWeight import build_fc_freqSum_NoTiedWeight_Small
from models.fc_freqSum_TiedWeight import build_fc_freqSum_NoTiedWeight_Medium
from models.fc_freqSum_TiedWeight import build_fc_freqSum_NoTiedWeight_Tiny
from models.fc_freqSum_TiedWeight import build_fc_freq_4_30_NoTiedWeight_Small
from models.fc_freqSum_TiedWeight import build_fc_freq_4_30_TiedWeight_Small
from models.fc_freqSum_TiedWeight import build_fc_freq_5_TiedWeight_Small

%load_ext autoreload
%autoreload 2

/data1/CHUONG_DATA/ChuongWork/Data4DeepLearning/src


In [24]:
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt 

In [7]:
def plot_feature(data, fName):
    plt.imshow(data, aspect='auto') 
    plt.colorbar() 
    plt.tight_layout() 
    plt.savefig(fName)
    plt.close() 

In [8]:
from argparse import Namespace

FLAGS = Namespace(
    model='freq_5_TiedWeight_Small',
    data_type = 'freqSum',
    data_normalization = 'normalize',
    data_base_dir = '/home/chuong/EEG-Project/processed_data',
    trained_model_base_dir = '/home/chuong/EEG-Project/trained_models',
    model_name = 'freq_5_TiedWeight_Small',
    trained_model_name = '2017-08-03-130758-relu',
    feature_activation = 'relu',
    output_base_dir = '',
)

In [9]:
 ##################################################################
# Read data 
print("{} and {}".format(FLAGS.model, FLAGS.data_base_dir))
sub_volumes_dir = get_input_data_path(FLAGS.model, FLAGS.data_base_dir)
eeg = eeg_subject_data()

if FLAGS.data_type == 'subsample': # subsample on 3D axes
    eeg.get_data(sub_volumes_dir, fake=FLAGS.test)
else: # no subsampling
    eeg.get_data(sub_volumes_dir, num_data_sec=-1, 
            fake=False, normalization=FLAGS.data_normalization)

subjects_data = eeg._data
subject_names = eeg._subjects


freq_5_TiedWeight_Small and /home/chuong/EEG-Project/processed_data
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...
Normalizing data ...


NameError: name 'subject_data' is not defined

In [11]:
X = subjects_data[0]
print('{} x {}'.format(X.shape[0], X.shape[1]))

subject_names

281 x 93575


['invcomp100Hz_sub30199_8remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30102_17remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30279_25remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30264_8remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30194_10remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30189_6remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30257_10remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30152_9remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30338_22remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30112_16remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30196_7remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30212_11remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30187_9remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30294_4remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30267_14remout3.mat_freq_5_cortexsample1.h5',
 'invcomp100Hz_sub30301_6remout3.mat_freq_5_cor

In [12]:
 # MODEL PATH
model_path = FLAGS.trained_model_base_dir + '/' + FLAGS.model  + '/' + FLAGS.trained_model_name
print("Model path: {}".format(model_path))
meta_files = glob.glob(model_path + '/*.meta')
meta_files.sort(key=os.path.getmtime)
meta_file_fullpath = meta_files[-1]


Model path: /home/chuong/EEG-Project/trained_models/freq_5_TiedWeight_Small/2017-08-03-130758-relu


In [13]:
#sess = tf.Session()
#saver = tf.train.import_meta_graph(meta_file_fullpath)
#check_point_dir = model_path + '/'    
#print("{}".format(check_point_dir))
#saver.restore(sess, tf.train.latest_checkpoint(check_point_dir))
#graph = tf.get_default_graph()

In [14]:
#[n.name for n in tf.get_default_graph().as_graph_def().node]

In [41]:
tf_version = tf.__version__.rpartition('.')[0]
with tf.Session() as sess:
    #if parse_version(tf_version) >= parse_version('0.12.0'):
    #    sess.run(tf.global_variables_initializer())
    #else:
    #    sess.run(tf.initialize_all_variables())
    # load the model
    saver = tf.train.import_meta_graph(meta_file_fullpath)
    check_point_dir = model_path + '/'
    print("{}".format(check_point_dir))
    saver.restore(sess, tf.train.latest_checkpoint(check_point_dir))
    
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name('input/Placeholder:0')
    dropout_keep_prob = graph.get_tensor_by_name('input/Placeholder_1:0')
    is_training = graph.get_tensor_by_name('input/Placeholder_2:0')
    feature_op = graph.get_tensor_by_name('FC4/Elu:0')
    
    output_features = []
    for subject_idx, subject_name in  enumerate(subject_names):
        subject_data = subjects_data[subject_idx]
        subject_time_features = []
        print("Working on {}".format(subject_name))
        #for time_idx in range(0, subject_data.shape[0]):
        #    batch_xs = subject_data[time_idx,:]
        batch_xs = subject_data
        feeds = {
                        x: batch_xs, 
                        dropout_keep_prob: 1.0, 
                        is_training: False
               }
        sub_feature = sess.run(feature_op, feed_dict=feeds)
        output_features.append(sub_feature)
        #subject_features = np.vstack(subject_time_features)
        #print("{:6.5f}".format(subject_feature))
        print("{} Feature shape: {}".format(subject_name, sub_feature.shape))

        #outFileName = subject_name + '.png'
        #plot_save_feature(subject_features, outFileName)


/home/chuong/EEG-Project/trained_models/freq_5_TiedWeight_Small/2017-08-03-130758-relu/
Working on invcomp100Hz_sub30199_8remout3.mat_freq_5_cortexsample1.h5
invcomp100Hz_sub30199_8remout3.mat_freq_5_cortexsample1.h5 Feature shape: (281, 64)
Working on invcomp100Hz_sub30102_17remout3.mat_freq_5_cortexsample1.h5
invcomp100Hz_sub30102_17remout3.mat_freq_5_cortexsample1.h5 Feature shape: (283, 64)
Working on invcomp100Hz_sub30279_25remout3.mat_freq_5_cortexsample1.h5
invcomp100Hz_sub30279_25remout3.mat_freq_5_cortexsample1.h5 Feature shape: (286, 64)
Working on invcomp100Hz_sub30264_8remout3.mat_freq_5_cortexsample1.h5
invcomp100Hz_sub30264_8remout3.mat_freq_5_cortexsample1.h5 Feature shape: (266, 64)
Working on invcomp100Hz_sub30194_10remout3.mat_freq_5_cortexsample1.h5
invcomp100Hz_sub30194_10remout3.mat_freq_5_cortexsample1.h5 Feature shape: (288, 64)
Working on invcomp100Hz_sub30189_6remout3.mat_freq_5_cortexsample1.h5
invcomp100Hz_sub30189_6remout3.mat_freq_5_cortexsample1.h5 Feature

40

In [44]:
for subject_idx, subject_name in enumerate(subject_names):
    subject_feat = output_features[subject_idx]
    plot_feature(subject_feat, subject_name + '.png')

In [45]:
pwd

u'/home/chuong/volumes_freq_5'

In [20]:
a.shape

(287, 64)

In [21]:
a.dtype

dtype('float32')